In [136]:
import numpy as np
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt

In [137]:
df = pl.read_csv('./nutrition.csv')
df.head()

index,name,serving_size,calories,total_fat,saturated_fat,cholesterol,sodium,choline,folate,folic_acid,niacin,pantothenic_acid,riboflavin,thiamin,vitamin_a,vitamin_a_rae,carotene_alpha,carotene_beta,cryptoxanthin_beta,lutein_zeaxanthin,lucopene,vitamin_b12,vitamin_b6,vitamin_c,vitamin_d,vitamin_e,tocopherol_alpha,vitamin_k,calcium,copper,irom,magnesium,manganese,phosphorous,potassium,selenium,...,arginine,aspartic_acid,cystine,glutamic_acid,glycine,histidine,hydroxyproline,isoleucine,leucine,lysine,methionine,phenylalanine,proline,serine,threonine,tryptophan,tyrosine,valine,carbohydrate,fiber,sugars,fructose,galactose,glucose,lactose,maltose,sucrose,fat,saturated_fatty_acids,monounsaturated_fatty_acids,polyunsaturated_fatty_acids,fatty_acids_total_trans,alcohol,ash,caffeine,theobromine,water
i64,str,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,...,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
0,"""Cornstarch""","""100 g""",381,"""0.1g""",null,"""0""","""9.00 mg""","""0.4 mg""","""0.00 mcg""","""0.00 mcg""","""0.000 mg""","""0.000 mg""","""0.000 mg""","""0.000 mg""","""0.00 IU""","""0.00 mcg""","""0.00 mcg""","""0.00 mcg""","""0.00 mcg""","""0.00 mcg""",0,"""0.00 mcg""","""0.000 mg""","""0.0 mg""","""0.00 IU""","""0.00 mg""","""0.00 mg""","""0.0 mcg""","""2.00 mg""","""0.050 mg""","""0.47 mg""","""3.00 mg""","""0.053 mg""","""13.00 mg""","""3.00 mg""","""2.8 mcg""",...,"""0.012 g""","""0.020 g""","""0.006 g""","""0.053 g""","""0.009 g""","""0.008 g""","""0""","""0.010 g""","""0.036 g""","""0.006 g""","""0.006 g""","""0.013 g""","""0.024 g""","""0.012 g""","""0.009 g""","""0.001 g""","""0.010 g""","""0.014 g""","""91.27 g""","""0.9 g""","""0.00 g""","""0""","""0""","""0""","""0""","""0""","""0""","""0.05 g""","""0.009 g""","""0.016 g""","""0.025 g""","""0.00 mg""","""0.0 g""","""0.09 g""","""0.00 mg""","""0.00 mg""","""8.32 g"""
1,"""Nuts, pecans""","""100 g""",691,"""72g""","""6.2g""","""0""","""0.00 mg""","""40.5 mg""","""22.00 mcg""","""0.00 mcg""","""1.167 mg""","""0.863 mg""","""0.130 mg""","""0.660 mg""","""56.00 IU""","""3.00 mcg""","""0.00 mcg""","""29.00 mcg""","""9.00 mcg""","""17.00 mcg""",0,"""0.00 mcg""","""0.210 mg""","""1.1 mg""","""0.00 IU""","""1.40 mg""","""1.40 mg""","""3.5 mcg""","""70.00 mg""","""1.200 mg""","""2.53 mg""","""121.00 mg""","""4.500 mg""","""277.00 mg""","""410.00 mg""","""3.8 mcg""",...,"""1.177 g""","""0.929 g""","""0.152 g""","""1.829 g""","""0.453 g""","""0.262 g""","""0""","""0.336 g""","""0.598 g""","""0.287 g""","""0.183 g""","""0.426 g""","""0.363 g""","""0.474 g""","""0.306 g""","""0.093 g""","""0.215 g""","""0.411 g""","""13.86 g""","""9.6 g""","""3.97 g""","""0.04 g""","""0""","""0.04 g""","""0.00 g""","""0.00 g""","""3.90 g""","""71.97 g""","""6.180 g""","""40.801 g""","""21.614 g""","""0.00 mg""","""0.0 g""","""1.49 g""","""0.00 mg""","""0.00 mg""","""3.52 g"""
2,"""Eggplant, raw""","""100 g""",25,"""0.2g""",null,"""0""","""2.00 mg""","""6.9 mg""","""22.00 mcg""","""0.00 mcg""","""0.649 mg""","""0.281 mg""","""0.037 mg""","""0.039 mg""","""23.00 IU""","""1.00 mcg""","""0.00 mcg""","""14.00 mcg""","""0.00 mcg""","""36.00 mcg""",0,"""0.00 mcg""","""0.084 mg""","""2.2 mg""","""0.00 IU""","""0.30 mg""","""0.30 mg""","""3.5 mcg""","""9.00 mg""","""0.081 mg""","""0.23 mg""","""14.00 mg""","""0.232 mg""","""24.00 mg""","""229.00 mg""","""0.3 mcg""",...,"""0.057 g""","""0.164 g""","""0.006 g""","""0.186 g""","""0.041 g""","""0.023 g""","""0""","""0.045 g""","""0.064 g""","""0.047 g""","""0.011 g""","""0.043 g""","""0.043 g""","""0.042 g""","""0.037 g""","""0.009 g""","""0.027 g""","""0.053 g""","""5.88 g""","""3.0 g""","""3.53 g""","""1.54 g""","""0""","""1.58 g""","""0""","""0""","""0.26 g""","""0.18 g""","""0.034 g""","""0.016 g""","""0.076 g""","""0.00 mg""","""0.0 g""","""0.66 g""","""0.00 mg"

In [138]:
df = df.select(
    [
        pl.col("name").alias("original_name"),
        pl.col("calories").alias("calories_per_100g"),
        pl.col("protein").str.extract('(\d*\.?\d+)').cast(float).fill_null(0.0).fill_nan(0.0).alias("protein"),
        pl.col("water").str.extract('(\d*\.?\d+)').cast(float).fill_null(0.0).fill_nan(0.0).alias("water"),
        pl.col("total_fat").str.extract('(\d*\.?\d+)').cast(float).fill_null(0.0).fill_nan(0.0).alias("fat"),
        pl.col("sugars").str.extract('(\d*\.?\d+)').cast(float).fill_null(0.0).fill_nan(0.0).alias("sugar"),
        pl.col("carbohydrate").str.extract('(\d*\.?\d+)').cast(float).fill_null(0.0).fill_nan(0.0).alias("carbohydrate"),
        pl.col("fiber").str.extract('(\d*\.?\d+)').cast(float).fill_null(0.0).fill_nan(0.0).alias("fiber"),
        pl.col("caffeine").str.extract('(\d*\.?\d+)').cast(float).fill_null(0.0).fill_nan(0.0).alias("caffeine"),
        pl.when(pl.col("alcohol").str.extract('(\d*\.?\d+)').cast(float).fill_null(0.0).fill_nan(0.0) >= 1E-8).then(True).otherwise(False).alias("contains_alcohol"),
    ]
)

In [139]:
df = df.select([pl.col("original_name").str.split(", ").arr.head(3).arr.join(" ").alias("name"), pl.all()]).unique(subset=["name"])

In [140]:
df.head()

name,original_name,calories_per_100g,protein,water,fat,sugar,carbohydrate,fiber,caffeine,contains_alcohol
str,str,i64,f64,f64,f64,f64,f64,f64,f64,bool
"""Cornstarch""","""Cornstarch""",381,0.26,8.32,0.1,0.0,91.27,0.9,0.0,false
"""Nuts pecans""","""Nuts, pecans""",691,9.17,3.52,72.0,3.97,13.86,9.6,0.0,false
"""Eggplant raw""","""Eggplant, raw""",25,0.98,92.3,0.2,3.53,5.88,3.0,0.0,false
"""Teff uncooked""","""Teff, uncooked...",367,13.3,8.82,2.4,1.84,73.13,8.0,0.0,false
"""Sherbet orange...","""Sherbet, orang...",144,1.1,66.1,2.0,24.32,30.4,1.3,0.0,false


In [141]:
from sqlalchemy import create_engine, Float, String, Boolean

engine = create_engine('sqlite://', echo=False)
df.to_pandas().to_sql(
    name='ingredients',
    con=engine,
    index=False,
    dtype={
        'original_name': String,
        'calories_per_100g': Float,
        'name': String,
        'protein': Float,
        'water': Float,
        'fat': Float,
        'sugar': Float,
        'carbohydrate': Float,
        'fiber': Float,
        'caffeine': Float,
        'contains_alcohol': Boolean,
    }
)

with engine.connect() as conn, open('./ingredient_initials.sql', 'w+') as f:
    for line in conn.connection.iterdump():
        f.write(line)
        f.write("\n")